# 8. Transfer learning. Rozpoznawanie obrazów

### Tomasz Rodak

Laboratorium 8

---

## 8.1

Celem tego laboratorium jest zapoznanie się z metodą [*transfer learningu*](https://en.wikipedia.org/wiki/Transfer_learning) (uczenia transferowego). Transfer learning to technika uczenia maszynowego, która polega na przenoszeniu wiedzy zdobytej podczas rozwiązywania jednego problemu do innego, pokrewnego problemu. W praktyce oznacza to, że zamiast trenować model od podstaw na dużym zbiorze danych, możemy wykorzystać model, który został już wytrenowany na innym zbiorze danych i dostosować go do naszych potrzeb. Jest to szczególnie przydatne w sytuacjach, gdy mamy ograniczone zasoby obliczeniowe lub mało danych do trenowania. 

Transfer learning zademonstrujemy na przykładzie problemu rozpoznawania obrazów. Wykorzystamy do tego celu pretrenowany model [ResNet18](https://arxiv.org/pdf/1512.03385). Obrazy do klasyfikacji pobierzemy z sekcji *Images* wyszukiwarki [DuckDuckGo](https://duckduckgo.com/). Wykorzystamy również bibliotekę [fastai](https://docs.fast.ai/) do budowy i trenowania modelu.

Arkusz ten powstał na podstawie kursu [fastai](https://course.fast.ai/), Getting Started

### 8.1.1 Instalacje i importy

Jeśli korzystasz z Google Colab, to biblioteka fastai jest już zainstalowana. Brakuje natomiast pakietu [`duckduckgo-search`](https://github.com/deedy5/duckduckgo_search). Pakiet ten wykorzystamy do utworzenia listy odnośników do obrazów, zainstalujesz go poleceniem:

In [ ]:
!pip install duckduckgo_search

Importy:

In [ ]:
import asyncio
import hashlib
import os
from io import BytesIO
from pathlib import Path

import httpx
import requests
import tqdm.notebook as tqdm
from duckduckgo_search import DDGS
from PIL import Image, UnidentifiedImageError

from fastai.vision.all import (
    DataBlock,
    ImageBlock,
    CategoryBlock,
    get_image_files,
    RandomSplitter,
    parent_label,
    Resize,
    resnet18, # Pretrenowana sieć konwolucyjna RESNET18
    vision_learner,
    error_rate,
    ClassificationInterpretation,
    PILImage
)

### 8.1.2 Pobieranie obrazów

Dla uproszczenia załóżmy, że chcemy rozpoznać dwa rodzaje obrazów, później możesz to oczywiście zmienić. Zaproponuj jakieś dwie kategorie. Dane będziemy pobierać z wyszukiwarki DuckDuckGo z sekcji *Images* wyszukiwania, więc uwzględnij wynikające stąd ograniczenia. Wybór bardzo specyficznej kategorii o słabej reprezentacji w internecie może skutkować nieadekwatnym zbiorem danych. Wybierz takie kategorie, o których spodziewasz się, że będą miały dobrą reprezentację w internecie. 

Funkcje `to_hash()` i `seen_files()` są pomocnicze. Funkcja `to_hash()` konwertuje ciąg znaków na hash MD5, a funkcja `seen_files()` pobiera zbiór nazw plików (bez rozszerzeń) z folderu. Użyjemy ich do sprawdzenia, czy dany obraz został już pobrany. Takie śledzenie plików jest istotne, gdyż różne zapytania do wyszukiwarki mogą czasami zwracać te same obrazy - nie chcemy ich pobierać więcej niż raz.

In [ ]:
def to_hash(s):
    """
    Konwertuje ciąg znaków na hash MD5.
    
    Argumenty:
        s (str): Ciąg znaków do hashowania
        
    Zwraca:
        str: Hash MD5 podanego ciągu w formacie szesnastkowym
    """
    return hashlib.md5(s.encode()).hexdigest()

def seen_files(folder):
    """
    Pobiera zbiór nazw plików (bez rozszerzeń) z folderu.
    
    Argumenty:
        folder (str lub Path): Ścieżka do folderu do przeskanowania
        
    Zwraca:
        set: Zbiór nazw plików bez rozszerzeń
    """
    files = os.listdir(folder)
    return set(Path(file).stem for file in files)

Tworzę folder `datasets/imgrec`, w którym będą przechowywane pobrane obrazy. Będę rozpoznawał obrazy dzika i jelenia. Aby je przechować, tworzę podfoldery `boar` i `deer`. Powinieneś wybrać inne kategorie, ale foldery powinny mieć podobną strukturę. Oznacza to, że powinieneś również dokonać odpowiednich zmian w kodzie poniżej, tak aby odpowiadał Twoim kategoriom. 

In [ ]:
# !mkdir -p datasets/imgrec
# !mkdir -p datasets/imgrec/boar
# !mkdir -p datasets/imgrec/deer

W tym miejscu tworzymy zapytanie do wyszukiwarki. **Upewnij się, że zapytanie jest odpowiednie do kategorii!** (zmienne `keywords` i `category`). Zbiór danych zostałby uszkodzony, gdybym przykładowo w kategorii `boar` dodał zapytanie o `deer`. W takim przypadku nie da się poprawnie wytrenować modelu.

Zapytanie delegowane jest do wyszukiwarki DuckDuckGo, która zwraca listę linków do obrazów (`images_links`). 

In [ ]:
# keywords = 'sus scrofa'
# category = 'boar'

with DDGS() as ddgs:
    images_gen = ddgs.images(
        keywords,
        max_results=100
    )
    images_links = list(images_gen)

Główna pętla pobierająca obrazy iteruje po wszystkich linkach i pobiera je asynchronicznie. Używamy `asyncio` do równoległego pobierania obrazów, co przyspiesza cały proces. Ważne zmienne:
- `FOLDER` - folder, w którym znajduje się zbiór danych
- `subfolder` - podfolder, w którym będą przechowywane pobrane obrazy w danej kategorii
- `already_seen` - zbiór nazw plików, które zostały już pobrane.

Nazwy plików są generowane na podstawie hasha MD5 adresu URL obrazu. Dzięki temu unikamy niespodziewanych znaków w nazwach plików. Funkcja `download_image_async()` sprawdza liczne możliwe błędy:
- `httpx.HTTPError` - błąd HTTP
- `httpx.TimeoutException` - przekroczony czas oczekiwania na odpowiedź
- `UnidentifiedImageError` - błąd związany z otwieraniem obrazu, dzięki czemu zapisujemy tylko poprawne obrazy.

Program ten możesz wykonywać wielokrotnie, pamiętaj jednak, aby wcześniej wstawić odpowiednie zapytanie i kategorię w komórce powyżej. 

In [ ]:
FOLDER = Path('datasets/imgrec')
subfolder = FOLDER/category
already_seen = seen_files(subfolder)
print(f'Now {len(already_seen)} files in {subfolder}')
counter = 0

async def download_image_async(client, image_url, already_seen, subfolder, to_hash, semaphore):
    async with semaphore:  # Limit concurrent downloads
        file = to_hash(image_url)

        if file in already_seen:
            print(f'Already seen: {file}={image_url}')
            return None

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:120.0) Gecko/20100101 Firefox/120.0'
            }
            r = await client.get(image_url, headers=headers, timeout=5)
        except httpx.HTTPError as e:
            print(f'HTTPError: {e} for {image_url}')
            return None
        except httpx.TimeoutException:
            print(f'Timeout: {image_url}')
            return None

        try:
            img = Image.open(BytesIO(r.content))
        except UnidentifiedImageError:
            print(f'Image error for {image_url}')
            return None

        file = Path(file).with_suffix(f'.{img.format.lower()}')
        print(f'New file = {image_url}')

        with open(subfolder/file, 'wb') as f:
            f.write(r.content)
            print(f'Saved {file} to {subfolder}')

        return file

max_concurrent = 10  
semaphore = asyncio.Semaphore(max_concurrent)

client = httpx.AsyncClient(limits=httpx.Limits(max_connections=20))

tasks = []
for image in images_links:
    image_url = image['image']
    task = download_image_async(client, image_url, already_seen, subfolder, to_hash, semaphore)
    tasks.append(task)

results = await asyncio.gather(*tasks)
await client.aclose()  # Close the client when done

successful = [r for r in results if r is not None]
print(f'Total files downloaded: {len(successful)}')

Rozmiar folderu `datasets/imgrec`:

In [ ]:
!du -hsc datasets/imgrec

Liczba pobranych obrazów w folderze `boar`:

In [ ]:
!ls -lh datasets/imgrec/boar/|wc -l

Liczba pobranych obrazów w folderze `deer`:

In [ ]:
!ls -lh datasets/imgrec/deer/|wc -l

### 8.1.3 Konstrukcja loadera danych

W tej sekcji tworzymy loader danych. Loader danych to obiekt, który umożliwia ładowanie danych w partiach (batchach) do modelu. W fastai używamy do tego celu klasy `DataBlock`:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=123),
    get_y=parent_label,
    item_tfms=[Resize(128, method='squish')]
).dataloaders(FOLDER, bs=16)

Argumenty z wywołania powyżej:

1. **Definicja bloków**  
   ```python
   blocks=(ImageBlock, CategoryBlock)
   ```  
   – pierwszy blok odpowiada za wczytywanie obrazów, drugi za etykiety (kategorie).

2. **Pobieranie plików**  
   ```python
   get_items=get_image_files
   ```  
   – wyszukuje w danym folderze wszystkie pliki graficzne.

3. **Podział na zbiory**  
   ```python
   splitter=RandomSplitter(valid_pct=0.2, seed=123)
   ```  
   – losowo dzieli dane na 80 % zbiór treningowy i 20 % walidacyjny, przy stałym ziarnie losowości.

4. **Pobieranie etykiet**  
   ```python
   get_y=parent_label
   ```  
   – etykieta dla każdego obrazu to nazwa katalogu, w którym się znajduje.

5. **Transformacja elementów**  
   ```python
   item_tfms=[Resize(128, method='squish')]
   ```  
   – zmiana rozmiaru każdego obrazu na 128×128 pikseli, bez zachowania proporcji (rozciągnięcie).

6. **Utworzenie DataLoaders**  
   ```python
   .dataloaders(FOLDER, bs=8)
   ```  
   – wskazanie ścieżki do głównego folderu z danymi i ustawienie rozmiaru batcha na 8.

Efektem jest obiekt `dls`, który udostępnia gotowe do użycia batch’e obrazów i odpowiadających im etykiet do trenowania i walidacji modelu.

Podgląd 6 losowo wybranych obrazów z batcha:

In [ ]:
dls.show_batch(max_n=6)

### 8.1.4 Budowa modelu

In [ ]:
learner = vision_learner(dls, resnet18, metrics=error_rate)

Ta komórka tworzy obiekt **Learner**, czyli gotowy do trenowania model CNN w Fastai:

1. **vision_learner**  
   – funkcja pomocnicza, która tworzy obiekt `Learner`.

2. **dls**  
   – przekazujemy wcześniej zdefiniowane `DataLoaders` (`dls`) ze zbiorami treningowym i walidacyjnym.

3. **resnet18**  
   – wykorzystujemy architekturę ResNet-18; domyślnie ładowana jest wstępnie wytrenowana sieć na ImageNet.

4. **metrics=error_rate**  
   – definiujemy metrykę do śledzenia podczas treningu – w tym wypadku odsetek błędnych klasyfikacji.

Efektem jest `learner`, który zawiera model, dane, optymalizator i wybraną metrykę, przygotowany do treningu i ewaluacji.

#### 8.1.5 Trening

In [ ]:
%%time
learner.fine_tune(3)

Powyższe wywołanie skutkuje transfer learningiem w dwóch fazach, rozłożonych na 3 epoki:

1. **Faza “rozgrzewki” (freeze)**  
   - Domyślnie przez **1 epokę** zamraża się wszystkie warstwy bazowego ResNet-a (oprócz głowy) i trenuje tylko głowę klasyfikatora,  

2. **Faza “dostrajania” (unfreeze)**  
   - Następnie wszystkie warstwy zostają odmrożone i trenujesz całą sieć przez pozostałe **2 epoki** (3 – 1 = 2) z automatycznie dobranym harmonogramem tempa uczenia

W efekcie `learner.fine_tune(3)` wykona:
- **1 epokę** treningu głowy przy zamrożonych wagach ResNet-a,  
- **2 epoki** treningu całej sieci po jej odmrożeniu.

Możesz zmienić liczbę epok fazy „rozgrzewki”, korzystając z dodatkowego argumentu `freeze_epochs`, np.:  
```python
learner.fine_tune(epochs=3, freeze_epochs=2)
```  
co da 2 epoki tylko głowy, a następnie 1 epokę pełnego odblokowania.

### 8.1.5 Ocena 

Macierz pomyłek:

In [ ]:
ci = ClassificationInterpretation.from_learner(learner)
ci.plot_confusion_matrix()

Podgląd 6 największych błędów klasyfikacji:

In [ ]:
ci.plot_top_losses(k=6)

Przykład klasyfikacji dla zupełnie nowych obrazów:

In [ ]:
# adres URL nowego obrazka; wstaw odpowiedni dla twojej kategorii
url = "https://as1.ftcdn.net/v2/jpg/01/27/10/72/1000_F_127107272_V6IH9cZhwyBMrKAmkomHAwGS5E8GTBwo.jpg"
r = requests.get(url)
r.raise_for_status()  # Sprawdź, czy żądanie zakończyło się sukcesem

# zapisz obrazek do pliku
with open('img.jpg', 'wb') as f:
    f.write(r.content)

In [ ]:
learner.predict(PILImage.create("img.jpg"))

In [ ]:
Image.open("./img.jpg")